In [2]:
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import torch.nn.utils.rnn as rnn_utils
DATA_PATH = "/Users/pranavsood/Documents/DDI/Gitlab/multi-modal-emotion/data"
SAVE_PATH = "/Users/pranavsood/Documents/DDI/Gitlab/multi-modal-emotion/src/run_scripts/save_files_from_run"


/Users/pranavsood/Documents/DDI/Gitlab/multi-modal-emotion/src/emotion_venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ORIG = {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger': 6}
NEW = {v: k for k, v in ORIG.items()} # reverse orig

In [9]:
def save_dataset(dataset) -> pd.DataFrame:
    """
    Get the dataset and perform some operations to process the dataset into what the code expects later on
    """
    df = pd.read_pickle(dataset)
    # print(df)
    

    """
    **REMOVED DIA125_UTT3 AS ITS NOT IN VAL DATASET**
    **REMOVED DIA110_UTT7 AS ITS NOT IN VAL DATASET**
    """


    folder_name = "data/"
    df.rename(columns = {'y':'emotion'}, inplace = True)
    df['path'] = folder_name + df['split'] + "_splits_wav/" + "dia" + df['dialog'].astype(str) + '_utt' + df['utterance'].astype(str) + ".wav"
    df['name'] = "dia" + df['dialog'].astype(str) + '_utt' + df['utterance'].astype(str) 
    # df.drop(columns = ["dialog" , "utterance" , "text"  , "num_words"] , inplace=True)
    df['label'] = df['emotion'].map(NEW)

    df = df[ df["name"] != "dia110_utt7"] 
    df = df[ df["name"] != "dia125_utt3"] 

    df.to_pickle(f"{DATA_PATH}/emotion_pd_raw.pkl")


    

In [10]:
save_dataset(f"{DATA_PATH}/emotion_pd.pkl")

In [14]:
emotion_raw = pd.read_pickle(f"{DATA_PATH}/emotion_pd_raw.pkl")


In [15]:
emotion_raw

,emotion,dialog,utterance,text,num_words,split,path,name,label
0,0,0,0,also i was the point person on my company s tr...,19,train,data/train_splits_wav/dia0_utt0.wav,dia0_utt0,neutral
1,0,0,1,you must ve had your hands full,7,train,data/train_splits_wav/dia0_utt1.wav,dia0_utt1,neutral
2,0,0,2,that i did that i did,6,train,data/train_splits_wav/dia0_utt2.wav,dia0_utt2,neutral
3,0,0,3,so let s talk a little bit about your duties,10,train,data/train_splits_wav/dia0_utt3.wav,dia0_utt3,neutral
4,0,0,5,"now you ll be heading a whole division , so yo...",17,train,data/train_splits_wav/dia0_utt5.wav,dia0_utt5,neutral
...,...,...,...,...,...,...,...,...,...
13703,6,279,2,you stole them from me !,6,test,data/test_splits_wav/dia279_utt2.wav,dia279_utt2,anger
13704,6,279,3,you stole them from me ! !,7,test,data/test_splits_wav/dia279_utt3.wav,dia279_utt3,anger
13705,6,279,4,gimme them !,3,test,data/test_splits_wav/dia279_utt4.wav,dia279_utt4,anger
13706,6,279,7,"look , i really need some help , okay ? why ? ...",29,test,data/test_splits_wav/dia279_utt7.wav,dia279_utt7,anger


In [16]:
labels = emotion_raw['emotion'].values 
        # Want a tensor of all the features d
audio_features = emotion_raw['path'].values

In [19]:
idx = 3
print(f"audio_features[idx] = {audio_features[idx]} \nlabels[idx] = {labels[idx]}")

audio_features[idx] = data/train_splits_wav/dia0_utt3.wav 
labels[idx] = 0


In [20]:
labels.shape

(13704,)

In [43]:
def tensorize(x):
    return torch.tensor( x, dtype=torch.float64)

In [23]:
def tensorizePandas(df):
    df['input_values']=df['input_values'].progress_apply(tensorize)
    df['labels']=df['labels'].progress_apply(tensorize)
    return df

In [26]:
# train_pd = tensorizePandas(pd.read_pickle(f"{SAVE_PATH}/train_pkl.pkl"))
# test_pd = tensorizePandas(pd.read_pickle(f"{SAVE_PATH}/test_pkl.pkl"))
val_pd = tensorizePandas(pd.read_pickle(f"{SAVE_PATH}/val_pkl.pkl"))

100%|██████████| 2608/2608 [00:00<00:00, 94114.54it/s]


In [27]:
all_pd = pd.concat([train_pd , test_pd , val_pd])

In [29]:
all_pd.to_pickle(f"{DATA_PATH}/emotion_audio_all_test.pkl")

In [30]:
all_pd['input_values'].values

array([tensor([-0.1474, -0.8100, -0.4489,  ...,  0.0037,  0.0014,  0.0014],
              dtype=torch.float64)                                         ,
       tensor([ 0.0158, -0.0453, -0.0712,  ..., -0.0026,  0.0034,  0.0019],
              dtype=torch.float64)                                         ,
       tensor([-0.0326, -0.0466, -0.0405,  ..., -0.0003, -0.0003, -0.0003],
              dtype=torch.float64)                                         ,
       ...,
       tensor([ 0.0524,  0.0480, -0.0150,  ..., -0.0008, -0.0002,  0.0004],
              dtype=torch.float64)                                         ,
       tensor([ 0.1020,  0.1000,  0.2102,  ..., -0.0003, -0.0003, -0.0003],
              dtype=torch.float64)                                         ,
       tensor([-0.0023, -0.0014, -0.0006,  ...,  0.0003,  0.0003, -0.0006],
              dtype=torch.float64)                                         ],
      dtype=object)

In [55]:
all_pd = pd.read_pickle(f"{DATA_PATH}/emotion_audio_all_test.pkl")

In [ ]:
def split(idx):
    idx = idx[5:7]
    if idx == "tr":
        return "train"
    if idx == "te":
        return "test"
    else:
        return "val"

In [27]:
all_pd['split'] = all_pd['path'].apply(split)

In [3]:
all_pd.to_pickle(f"{DATA_PATH}/emotion_audio_all.pkl")

In [4]:
x = all_pd['input_values'].values

In [6]:
x

array([array([-0.14737721, -0.80998605, -0.4489204 , ...,  0.00367516,
               0.00142903,  0.00142903], dtype=float32)               ,
       array([ 0.01583829, -0.04533609, -0.07119843, ..., -0.00256376,
               0.00340448,  0.00191242], dtype=float32)               ,
       array([-0.03260687, -0.04656146, -0.04045632, ..., -0.0003369 ,
              -0.0003369 , -0.0003369 ], dtype=float32)               ,
       ...,
       array([ 0.05236564,  0.04803834, -0.01501651, ..., -0.00079826,
              -0.00018007,  0.00043811], dtype=float32)               ,
       array([ 0.10200054,  0.10001467,  0.21023032, ..., -0.00027165,
              -0.00027165, -0.00027165], dtype=float32)               ,
       array([-0.00232242, -0.00143894, -0.00055545, ...,  0.00032803,
               0.00032803, -0.00055545], dtype=float32)               ],
      dtype=object)

In [3]:
torch.from_numpy(all_pd['input_values'].values)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [6]:
def tensorize(x):
    return torch.tensor( x, dtype=torch.float64)

In [4]:
def boolean_indexing(v):
    lens = np.array([len(item) for item in v])
    mask = lens[:,None] > np.arange(lens.max())
    out = np.zeros(mask.shape,dtype=int)
    out[mask] = np.concatenate(v)
    return out

In [ ]:
y = boolean_indexing(all_pd['input_values'].values)
type(y)
len(y[0])

Error: Session cannot generate requests

In [5]:
# torch.tensor(all_pd['input_values'].values[0] , dtype=torch.float64)



tqdm.pandas()
all_pd['input_values'] = all_pd['input_values'].progress_apply(tensorize)
all_pd['labels'] = all_pd['labels'].progress_apply(tensorize)


100%|██████████| 13704/13704 [02:26<00:00, 93.54it/s] 


In [46]:
all_pd.to_pickle(f"{DATA_PATH}/emotion_audio_all.pkl")

In [32]:
import random as r
all_pd.iloc[r.randint(0,1300)]['input_values'].shape

torch.Size([1230846])

In [ ]:
print(all_pd['input_values'].values[0:5])
print(all_pd[['labels']].to_numpy()[0:5])


In [38]:
batch = all_pd['input_values'].values[0:5]

In [40]:
result = rnn_utils.pad_sequence(batch, batch_first=True)
for tens in result:
    print(tens.shape)

torch.Size([181590])
torch.Size([181590])
torch.Size([181590])
torch.Size([181590])
torch.Size([181590])


In [1]:
import torchaudio
def speech_file_to_array_fn(path , target_sampling_rate):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

/Users/pranavsood/Documents/DDI/Gitlab/multi-modal-emotion/src/emotion_venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from numpy import array
batch = [('data/train_splits_wav/dia7_utt0.wav', array(0)), ('data/train_splits_wav/dia8_utt0.wav', array(0)), ('data/train_splits_wav/dia8_utt1.wav', array(0)), ('data/train_splits_wav/dia8_utt2.wav', array(0)), ('data/train_splits_wav/dia8_utt3.wav', array(0)), ('data/train_splits_wav/dia8_utt5.wav', array(0)), ('data/train_splits_wav/dia9_utt0.wav', array(0)), ('data/train_splits_wav/dia9_utt1.wav', array(0))]

In [4]:
len(batch)

8

In [3]:
from transformers import Wav2Vec2Processor
speech_list = []
label_list = []
# pdb.set_trace()


model_path = "facebook/wav2vec2-large-960h"
processor = Wav2Vec2Processor.from_pretrained(model_path)
target_sampling_rate = processor.feature_extractor.sampling_rate

for (input_path , label) in batch:
    speech_list.append(speech_file_to_array_fn("/Users/pranavsood/Documents/DDI/Gitlab/multi-modal-emotion/" + input_path , target_sampling_rate))
    label_list.append(label)

In [5]:
type(speech_list[0]) # speech list is list of numpy arrays

numpy.ndarray

In [23]:
import numpy as np

m = max(map(np.shape , speech_list))[1]
m

117760

In [21]:
a = speech_list[0]
a.shape

(6, 46080)

In [25]:
6*46080

276480

In [45]:
# np.resize(a , (6,m)).shape
m = max(map(np.shape , speech_list))[1]
lol = [   np.resize(matrix , (6,m)) for matrix in speech_list  ] # list of tensors
# lol = [   torch.Tensor(np.resize(matrix , (6,m))) for matrix in speech_list  ] # list of tensors

In [46]:
result = processor(lol, sampling_rate=target_sampling_rate , padding = True)#["input_values"][0]

In [52]:
result['input_values'] # list of np array of np arrays 

numpy.ndarray

In [53]:
ret = [torch.Tensor(matrix) for matrix in result['input_values']]

In [5]:
x = np.array([[1,2] , [3 , 4] , [5 , 6] , [7 , 8] , [9,10] , [11,12]])
x

array([[ 1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 7,  8],
       [ 9, 10],
       [11, 12]])

In [27]:
f = np.random.rand(6,3)
f

array([[0.92693849, 0.82481117, 0.921607  ],
       [0.57440081, 0.88573636, 0.19654986],
       [0.09443722, 0.48478423, 0.18508989],
       [0.76760865, 0.15714948, 0.7863017 ],
       [0.8227047 , 0.60155623, 0.44553038],
       [0.24566853, 0.51130251, 0.55881275]])

In [28]:
lst = [x , f]

In [30]:
m = max(map(np.shape , lst))[1]
inp = []
for matrix in lst:
    n = matrix.shape[1]
    mat = np.resize(matrix , (1,m*6))[0]
    mat[n*6 : m*6] = 0
    inp.append(mat)
np.array(inp).shape

(2, 18)

In [26]:
x.shape

(6, 2)

In [25]:
for mat in x:
    print(mat.shape)
    # mat = mat*2
x

(2,)
(2,)
(2,)
(2,)
(2,)
(2,)


array([[ 1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 7,  8],
       [ 9, 10],
       [11, 12]])

In [15]:
y = np.resize(x , (1, 18))[0]
y

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  5,
        6])

In [21]:
y[6*2:6*3] = 0

In [22]:
y

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  0,  0,  0,  0,  0,
        0])